In [1]:
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
Updating files:  65% (1620/2466)
Updating files:  66% (1628/2466)
Updating files:  67% (1653/2466)
Updating files:  68% (1677/2466)
Updating files:  69% (1702/2466)
Updating files:  70% (1727/2466)
Updating files:  71% (1751/2466)
Updating files:  72% (1776/2466)
Updating files:  73% (1801/2466)
Updating files:  74% (1825/2466)
Updating files:  75% (1850/2466)
Updating files:  76% (1875/2466)
Updating files:  77% (1899/2466)
Updating files:  78% (1924/2466)
Updating files:  79% (1949/2466)
Updating files:  80% (1973/2466)
Updating files:  81% (1998/2466)
Updating files:  82% (2023/2466)
Updating files:  82% (2026/2466)
Updating files:  83% (2047/2466)
Updating files:  84% (2072/2466)
Updating files:  85% (2097/2466)
Updating files:  86% (2121/2466)
Updating files:  87% (2146/2466)
Updating files:  88% (2171/2466)
Updating files:  89% (2195/2466)
Updating files:  90% (2220/2466)
Updating files:  91% (2245/2466)
Updating files:  92% (2269/2466)
Updating files:  9

# Importamos librerías necesarias

In [2]:
import os
import six.moves.urllib as urllib
import sys
import tarfile
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt

from IPython.display import display
import cv2
from PIL import Image
import numpy as np



#From google
import tensorflow as tf
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util


# Patch the location of gfile
tf.gfile = tf.io.gfile

In [4]:
def load_model(model_name):

    model_file ='http://download.tensorflow.org/models/object_detection/'+ model_name + '.tar.gz'

    model_dir = tf.keras.utils.get_file(fname=model_name,origin=model_file,untar=True)

    model_dir = pathlib.Path(model_dir)/"saved_model"

    model = tf.saved_model.load(str(model_dir)) 

    model = model.signatures['serving_default']

    return model

def run_inference_for_single_image(model, image):

    input_tensor = tf.convert_to_tensor(image)
    input_tensor = input_tensor[tf.newaxis,...]


    output_dict = model(input_tensor)
    num_detections = int(output_dict.pop('num_detections'))

    for key,value in output_dict.items():    
        output_dict[key]= value[0,:num_detections].numpy()

    output_dict['num_detections'] = num_detections

    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

    return output_dict

# Carga de modelo y etiquetas

In [5]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [6]:
model_name = 'ssdlite_mobilenet_v2_coco_2018_05_09'
detection_model = load_model(model_name)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


# Para una sola imagen

In [8]:
img = cv2.imread('escritorio.jpg')
output_dict = run_inference_for_single_image(detection_model, img)

In [9]:
temp=vis_util.visualize_boxes_and_labels_on_image_array(
        img,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks_reframed', None),
        use_normalized_coordinates=True,line_thickness=8)

In [10]:
cv2.namedWindow('image')
cv2.imshow('image',temp)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Para un streaming local de video

In [11]:
cap = cv2.VideoCapture(0)

while(cap.isOpened()):

    ret , image_np = cap.read()  

    output_dict = run_inference_for_single_image(detection_model, image_np)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks_reframed', None),
        use_normalized_coordinates=True,line_thickness=8)

    if ret == True:
        
        cv2.imshow('frame',image_np)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()
